# Room Locator (BIG Model)

###  Import TensorFlow 

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import mixed_precision
from matplotlib import pyplot as plt 

# Enable mixed precision for speed and lower memory on M-series GPUs
mixed_precision.set_global_policy("mixed_float16")

### Load Image Dataset

In [3]:
DATA_DIR = "Dataset"
IMG_SIZE = 456
BATCH_SIZE = 64
NUM_CLASSES = 27

AUTOTUNE = tf.data.AUTOTUNE

train_ds = keras.utils.image_dataset_from_directory(
    DATA_DIR,
    labels="inferred",
    label_mode="categorical",
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset="training",
)

val_ds = keras.utils.image_dataset_from_directory(
    DATA_DIR,
    labels="inferred",
    label_mode="categorical",
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset="validation",
)

# Pipeline optimizations
train_ds = train_ds.prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)

Found 6103 files belonging to 27 classes.
Using 4883 files for training.


2026-01-15 11:25:43.952733: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2026-01-15 11:25:43.952755: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2026-01-15 11:25:43.952762: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.66 GB
I0000 00:00:1768472743.953266 41449788 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1768472743.953333 41449788 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Found 6103 files belonging to 27 classes.
Using 1220 files for validation.


### Add Noise / Randomness

In [4]:
# Data augmentation tuned for location invariance
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.12),
        layers.RandomZoom(0.15),
        layers.RandomTranslation(0.1, 0.1),
        layers.RandomContrast(0.2),
        layers.RandomBrightness(0.2),
    ],
    name="data_augmentation",
)

### Setup and Build Training Model

In [5]:
# Base model: EfficientNetB5 for higher capacity
base_model = keras.applications.EfficientNetB5(
    include_top=False,
    weights="imagenet",
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
)

# Stage 1: freeze backbone
base_model.trainable = False

inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = data_augmentation(inputs)
x = keras.applications.efficientnet.preprocess_input(x)

x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D(name="global_avg_pool")(x)

# Lighter classification head with regularization
x = layers.BatchNormalization()(x)
x = layers.Dense(512, activation="relu", kernel_regularizer=keras.regularizers.l2(1e-4))(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(256, activation="relu", kernel_regularizer=keras.regularizers.l2(1e-4))(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax", dtype="float32", name="predictions")(x)

model = keras.Model(inputs, outputs, name="efficientnetB5_tum_locations")

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.05),
    metrics=[
        "accuracy",
        keras.metrics.TopKCategoricalAccuracy(k=3, name="top_3_acc"),
    ],
)

### Execute Training Model

In [8]:
# Show the model structure and run a short training loop so the cell produces output
model.summary()
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=40,  # increase after verifying things run
    verbose=1,
 )

Model: "efficientnetB5_tum_locations"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 456, 456, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 456, 456, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb5 (Functional)     │ (None, 15, 15, 2048)   │    28,513,527 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_avg_pool                 │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 2048)           │         8,192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ predictions (Dense)             │ (None, 27)             │         6,939 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,091,982 (122.42 MB)

 Trainable params: 1,191,451 (4.55 MB)

 Non-trainable params: 28,517,623 (108.79 MB)

 Optimizer params: 2,382,908 (9.09 MB)

Epoch 1/40
77/77 ━━━━━━━━━━━━━━━━━━━━ 532s 7s/step - accuracy: 0.5916 - loss: 3.0122 - top_3_acc: 0.8489 - val_accuracy: 0.7189 - val_loss: 1.3739 - val_top_3_acc: 0.9418
Epoch 2/40
77/77 ━━━━━━━━━━━━━━━━━━━━ 526s 7s/step - accuracy: 0.6019 - loss: 2.8519 - top_3_acc: 0.8550 - val_accuracy: 0.7328 - val_loss: 1.4893 - val_top_3_acc: 0.9475
Epoch 3/40
77/77 ━━━━━━━━━━━━━━━━━━━━ 514s 7s/step - accuracy: 0.6005 - loss: 2.7676 - top_3_acc: 0.8583 - val_accuracy: 0.7213 - val_loss: 1.7020 - val_top_3_acc: 0.9311
Epoch 4/40
77/77 ━━━━━━━━━━━━━━━━━━━━ 511s 7s/step - accuracy: 0.6279 - loss: 2.6338 - top_3_acc: 0.8661 - val_accuracy: 0.7303 - val_loss: 1.7083 - val_top_3_acc: 0.9459
Epoch 5/40
77/77 ━━━━━━━━━━━━━━━━━━━━ 503s 7s/step - accuracy: 0.6295 - loss: 2.5300 - top_3_acc: 0.8700 - val_accuracy: 0.7402 - val_loss: 1.6908 - val_top_3_acc: 0.9451
Epoch 6/40
77/77 ━━━━━━━━━━━━━━━━━━━━ 507s 7s/step - accuracy: 0.6336 - loss: 2.4424 - top_3_acc: 0.8781 - val_accuracy: 0.7598 - val_loss: 1.641

### Plot Epocs

In [6]:
plt.plot(history.epoch, history.history["loss"])
plt.title("Train Loss Curve")
plt.show()

plt.plot(history.epoch, history.history["accuracy"])
plt.title("Train Accuracy Curve")
plt.show()

NameError: name 'history' is not defined

### Fine-Tuning Model

In [ ]:
# Unfreeze top blocks of EfficientNetB5 for fine-tuning
base_model.trainable = True

fine_tune_at = len(base_model.layers) * 3 // 4  # top ~25% of layers

for i, layer in enumerate(base_model.layers):
    layer.trainable = i >= fine_tune_at
    # Keep BatchNorm in eval mode for stability
    if isinstance(layer, layers.BatchNormalization):
        layer.trainable = False

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.05),
    metrics=[
        "accuracy",
        keras.metrics.TopKCategoricalAccuracy(k=3, name="top_3_acc"),
    ],
)

# Then train again with early stopping + LR schedule
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=3,
    restore_best_weights=True,
 )